In [1]:
def my_decorator(func):
    def wrapper():
        print("Что-то происходит ДО вызова функции")
        func()
        print("Что-то происходит ПОСЛЕ вызова функции")

    return wrapper


@my_decorator
def say_hello():
    print("Привет!")


say_hello()

Что-то происходит ДО вызова функции
Привет!
Что-то происходит ПОСЛЕ вызова функции


In [2]:
def concatenate(a, b):
    a
    return a + b

In [11]:
import tkinter as tk
from tkinter import ttk


class AppButton:
    def __init__(self, master, text, command, font, width, height, bg, fg, pack_options=None):
        self.btn = tk.Button(
            master,
            text=text,
            font=font,
            width=width,
            height=height,
            bg=bg,
            fg=fg,
            command=command
        )
        # Если pack_options не указан — используем дефолтные
        pack_options = pack_options or {"pady": 5}
        self.btn.pack(**pack_options)


class AppTable:
    def __init__(self, master, columns, show, height):
        self.frame = tk.Frame(master, height=height)
        self.frame.pack(fill=tk.X, padx=2, pady=2)
        self.frame.pack_propagate(False)  # Фиксируем размер Frame

        self.table = ttk.Treeview(
            self.frame,  # <-- внутри frame
            columns=columns,
            show=show
        )
        self.table.pack(fill=tk.BOTH, expand=True, padx=10, pady=10)

        for col in columns:
            self.table.heading(col, text=col)
            self.table.column(col, width=170)


class App:
    def __init__(self, root):
        self.root = root
        self.root.title('Телефонный справочник')
        self.root.geometry("1000x850")
        self.root.configure(bg='black')
        self.show_start_screen()


    def clear_screen(self):
        # Очистка окна
        for widget in self.root.winfo_children():
            widget.destroy()

    def show_start_screen(self):

        self.clear_screen()

        buttons = [
            ("Показать все контакты", self.get_all_contacts ),
            ("Создать контакт", self.create_contact),
            ("Найти контакт", self.search_contact),
            ("Изменить контакт", self.edit_contact),
            ("Выход", self.root.destroy),
        ]

        for text, command in buttons:
            if text == "Выход":
                bg_color = "red"
            else:
                bg_color = "black"

            AppButton(
                self.root,
                text=text,
                font=("Courier New", 15, "bold"),
                width=75,
                height=7,
                bg=bg_color,
                fg="white",
                command=command
            )

    def get_all_contacts(self):
        from functional import get_all_contacts  # Чтобы избежать циклического импорта
        get_all_contacts(self.root, self)   # Передаем self (экземпляр App)

    def create_contact(self):
        pass

    def search_contact(self):
        pass

    def edit_contact(self):
        pass




ValueError: invalid literal for int() with base 10: ''

In [12]:
import uuid
import tkinter as tk
from utils import load_contacts, write_contacts
from GUI import AppButton, AppTable


def get_all_contacts(root, App):
    App.clear_screen()

    columns = ("ID", "Имя", "Телефон", "Email", "Комментарий")
    tree = AppTable(root, columns=columns, show="headings", height=800)

    contacts = load_contacts()
    # Добавление данных в таблицу
    for contact in contacts:
        tree.table.insert(
            "",
            tk.END,
            values=(
                contact["ID"],
                contact["Имя"],
                contact["Телефон"],
                contact["Email"],
                contact["Комментарий"],
            ),
        )

    AppButton(
        root,
        text="Назад",
        font=("Arial", 12),
        width=15,  # можно подобрать значение под нужный размер
        height=2,  # высота в строках
        bg="red",
        fg="white",
        command=App.show_start_screen,
    )


# Заглушки для будущих функций
def create_contact(self):
    pass


def search_contact(self):
    pass


def edit_contact(self):
    pass


# Функция добавления нового контакта
def add_contact(name, phone, email, comment):
    contacts = load_contacts()
    new_contact = {
        "ID": str(uuid.uuid4()),  # Уникальный ID
        "Имя": name,
        "Телефон": phone,
        "Email": email,
        "Комментарий": comment,
    }
    contacts.append(new_contact)
    write_contacts(contacts)
    return new_contact

д
